### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [2]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

Note: you may need to restart the kernel to use updated packages.


### Load data and model

In [3]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [77]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name).to("mps")

In [16]:
tokenizer(qqp['train'][3]["text1"], qqp['train'][3]["text2"])

{'input_ids': [0, 2264, 64, 65, 109, 71, 17025, 3297, 116, 2, 2, 2264, 109, 939, 109, 71, 127, 17025, 3297, 17487, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenizer(qqp['train'][4]["text1"])

{'input_ids': [0, 13841, 64, 38, 465, 10, 476, 9455, 13, 127, 9972, 23, 5703, 4414, 116, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2' ],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map: 100%|██████████| 40430/40430 [00:01<00:00, 36463.47 examples/s]


In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[0, 6179, 16, 5, 301, 9, 10, 10638, 1294, 116, 9918, 47, 6190, 110, 308, 3734, 116, 2, 2, 32251, 672 ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [6]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=3, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=3
)

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [26]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0, 0, 1]), 'idx': tensor([0, 1, 2]), 'input_ids': tensor([[    0,  7608,    32,  1704,    12, 15085,    98,  2721,   116,     2,
             2,  7608,    32,    39, 12560,  2857,    98,  2721,   116,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     

TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [8]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=256, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=2,
)

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
from torchmetrics.classification import Accuracy
answers = []
preds = []
accuracy = Accuracy(task="multiclass", num_classes=2).to("mps")
with torch.no_grad():
    for batch in val_loader:
      
        predicted = model(
        input_ids=batch['input_ids'].to("mps"),
        attention_mask=batch['attention_mask'].to("mps"),
        token_type_ids=batch['token_type_ids'].to("mps"),
        )
        preds.append(torch.argmax(predicted.logits, dim = -1))
        answers.append(batch["labels"].to("mps"))
    


preds = torch.cat(preds)
answers = torch.cat(answers)
    

accuracy =  accuracy(preds, answers)


In [11]:
accuracy

tensor(0.3516, device='mps:0')

In [110]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model =  AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base").to("mps")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torchmetrics.classification import Accuracy
answers = []
preds = []
accuracy = Accuracy(task="multiclass", num_classes=2).to("mps")
with torch.no_grad():
    for batch in val_loader:
      
        predicted = model(
        input_ids=batch['input_ids'].to("mps"),
        attention_mask=batch['attention_mask'].to("mps"),
        )
        preds.append(torch.argmax(predicted.logits, dim = -1))
        answers.append(batch["labels"].to("mps"))
        break
    
preds = torch.cat(preds)
answers = torch.cat(answers)
    
accuracy =  accuracy(preds, answers)


In [ ]:
predicted.logits

In [ ]:

import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import pandas as pd
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

class lightning_module(L.LightningModule):
   
    def __init__(self, bert, tokenizer):
      super().__init__()
      self.bert = bert
      self.tokenizer = tokenizer
      self.accuracy = Accuracy(task="multiclass", num_classes=2).to("mps")

      for name, param in self.bert.base_model.named_parameters():
          param.requires_grad = False

      for name, param in self.bert.base_model.named_parameters():
          if "pooler" in name:
              param.requires_grad = True


    def forward(self, batch):
    
       return  self.bert(input_ids=batch['input_ids'].to("mps"),
        attention_mask=batch['attention_mask'].to("mps"),)

    def training_step(self, batch):
      
      model_output = self(batch)
      loss = F.cross_entropy(model_output.logits, batch["labels"])
      self.log("train_loss", loss)
      predicts = torch.argmax(model_output.logits, dim = -1) 
      accuracy = self.accuracy(predicts, batch["labels"])
      self.log("train_accuracy", accuracy)
      return loss
    
    def validation_step(self, batch):
      model_output = self(batch)
      loss = F.cross_entropy(model_output.logits, batch["labels"])
      self.log("val_loss", loss)
      predicts = torch.argmax(model_output.logits, dim = -1) 
      accuracy = self.accuracy(predicts, batch["labels"])
      self.log("val_accuracy", accuracy)
      return loss
    
    def configure_optimizers(self):
       optimizer = torch.optim.Adam(self.parameters(), lr=5e-5) 
       return optimizer
    
model_lightning = lightning_module(model, tokenizer)


checkpoint_callback = ModelCheckpoint(
   dirpath="checkpoints",
   filename="best-checkpoint",
   save_top_k=1,
   verbose= True,
   monitor="val_loss",
   mode = "min"
)

stopping = EarlyStopping(monitor="val_loss", mode="min", patience=2)
logger = TensorBoardLogger("ligtning_logs", name = "surface")

trainer = Trainer(logger=logger ,max_epochs=15 , accelerator="mps", enable_progress_bar=True, callbacks=[checkpoint_callback, stopping])

val_loader = torch.utils.data.DataLoader(
    qqp_preprocessed['validation'], batch_size=256, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=2,
)
train_loader = torch.utils.data.DataLoader(
     qqp_preprocessed['train'], batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=2,
)

trainer.fit(model_lightning, train_loader, val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name     | Type                             | Params | Mode 
----------------------------------------------------------------------
0 | bert     | DebertaForSequenceClassification | 139 M  | eval 
1 | accuracy | MulticlassAccuracy               | 0      | train
----------------------------------------------------------------------
592 K     Trainable params
138 M     Non-trainable params
139 M     Total params
556.775   Total estimated model params size (MB)
1         Modules in train mode
242       Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 27.02 GB, other allocations: 1.78 MB, max allowed: 27.20 GB). Tried to allocate 192.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.